# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [2]:
import pandas as pd
import numpy as np
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
insurance = pd.read_csv(url)

In [34]:
insurance["Total Claim Amount"] = insurance["Total Claim Amount"].astype("int64")
new_df = insurance[(insurance["Response"]=="Yes") & (insurance["Total Claim Amount"]>1000)]

In [61]:
insurance.head(3)

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480,SUV,Medsize,A


In [66]:
insurance["Number of Policies"].value_counts()

Number of Policies
1    3864
2    2748
3    1380
7     532
9     505
5     498
4     488
8     461
6     434
Name: count, dtype: int64

In [36]:
insurance.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [52]:
yes_peeps = insurance[(insurance["Response"]=="Yes")]
yes_peeps.groupby(["Policy Type","Gender"])[("Total Claim Amount")].mean().round(2)

Policy Type     Gender
Corporate Auto  F         433.30
                M         408.12
Personal Auto   F         452.50
                M         456.58
Special Auto    F         452.86
                M         429.12
Name: Total Claim Amount, dtype: float64

In [67]:
insurance.duplicated().sum()

0

In [69]:
insurance.groupby(["Customer"])["State"].size().head(3)

Customer
AA10041    3
AA11235    2
AA16582    1
Name: State, dtype: int64

In [60]:
# 3. Analyze the total number of customers who have policies in each state,
# and then filter the results to only include states where there are more than 500 customers.
states = ["California","Oregon","Arizona","Nevada","Washington"]
insurance.groupby(["State"])["Customer"].size()

State
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: Customer, dtype: int64

In [79]:
# 4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
insurance.groupby(["Education","Gender"])["Customer Lifetime Value"].describe().sort_values(by="mean")

count         mean          std          min  \
Education            Gender                                                  
Doctor               F        200.0  7328.508916  6930.556300  2395.570000   
                     M        201.0  7415.333638  5460.941400  2267.604038   
Bachelor             M       1572.0  7703.601675  6658.460774  1898.007675   
College              F       1608.0  7748.823325  6678.304057  1898.683686   
Bachelor             F       1700.0  7874.269478  6614.107319  1904.000852   
College              M       1596.0  8052.459288  6716.960507  1918.119700   
High School or Below M       1556.0  8149.687783  7190.619899  1940.981221   
Master               F        475.0  8157.053154  6621.774653  2417.777032   
                     M        412.0  8168.832659  7362.156626  2272.307310   
High School or Below F       1590.0  8675.220201  7497.848501  2144.921535   

                                     25%          50%          75%  \
Education            Gender                                          
Doctor               F       3531.281803  5332.462694  7662.703676   
                     M       3557.368368  5577.669457  8828.380695   
Bachelor             M       3842.347006  5548.031892  8646.695783   
College              F       3866.801886  5623.611187  8714.922135   
Bachelor             F       4118.588554  5640.505303  8723.239761   
College              M       4035.902621  6005.847375  9157.757714   
High School or Below M       4197.681981  6286.731006  9002.190432   
Master               F       4061.874865  5729.855012  9264.035235   
                     M       3933.033944  5579.099207  9031.077409   
High School or Below F       4158.278110  6039.553187  9457.340070   

                                     max  
Education            Gender               
Doctor               F       44856.11397  
                     M       32677.34284  
Bachelor             M       67907.27050  
College              F       61850.18803  
Bachelor             F       73225.95652  
College              M       61134.68307  
High School or Below M       83325.38119  
Master               F       51016.06704  
                     M       50568.25912  
High School or Below F       55277.44589

In [ ]:
"""
On average customers that belong to the educational level of 'high school or below' have the highest lifetime value. 
"""

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here